In [38]:
import numpy as np

class Bootstrap():
    def __init__(self, X, y, n_samples, len_sample):
        self.n_samples = n_samples
        self.len_sample = len_sample
        self.X = np.array(X)
        self.y = np.array(y)
        self.sample_X = np.zeros((n_samples, len_sample, self.X.shape[1]))
        self.sample_y = np.zeros((n_samples, len_sample, self.y.shape[1]))
        for i in range(n_samples):
            idx = np.random.choice(np.arange(len(X)), len_sample, replace=True)
            self.sample_X[i] = self.X[idx]
            self.sample_y[i] = self.y[idx]
        print('a', *self.sample_X)
        print('b', *self.sample_y)
    def eval_model_error(model, error):
        result = np.zeros(n_samples)
        for i in range(n_samples):
            result[i] = error(model, self.sample_X[i], self.sample_y[i])
        return result
    def fit_eval_model_error(model, error):
        result = np.zeros(n_samples)
        for i in range(n_samples):
            model.fit(self.sample_X[i], self.sample_y[i])
            result[i] = error(model, self.sample_X[i], self.sample_y[i])
        return result
        


In [39]:
X = [[1, 2], [3, 4], [5, 6]]
b = Bootstrap(X, X, 5, 3)


a [[ 3.  4.]
 [ 1.  2.]
 [ 3.  4.]] [[ 3.  4.]
 [ 1.  2.]
 [ 1.  2.]] [[ 3.  4.]
 [ 3.  4.]
 [ 3.  4.]] [[ 5.  6.]
 [ 1.  2.]
 [ 5.  6.]] [[ 1.  2.]
 [ 3.  4.]
 [ 5.  6.]]
b [[ 3.  4.]
 [ 1.  2.]
 [ 3.  4.]] [[ 3.  4.]
 [ 1.  2.]
 [ 1.  2.]] [[ 3.  4.]
 [ 3.  4.]
 [ 3.  4.]] [[ 5.  6.]
 [ 1.  2.]
 [ 5.  6.]] [[ 1.  2.]
 [ 3.  4.]
 [ 5.  6.]]


In [40]:
class Model():
    def __init__(self):
        pass
    def fit(self, X, y):
        pass
    def predict(self, X):
        pass
    def predict_params(self, X):
        pass
        

In [43]:
class Metric():
    def __init__(model):
        self.model = model
    def eval(X, y):
        return 0
        pass
    

In [44]:
class EvaluateInfo():
    def __init__(model, metric):
        self.model = model
        self.metric = metric # a function from X and y
    def fit(X, y, n_samples=20,len_sample=None, typ='static'):
        if len_sample is None:
            len_sample = X.shape[1]
        b = Bootstrap(X, y, n_samples, len_sample)
        if typ == 'static':
            model_metric.fit(X, y)
            self.errors = b.eval_model_error(model, metric)
        else:
            self.errors = b.fit_eval_model_errr(model, metric)
        pass
    #def plot():
        #plt.figure(figsize=(15, 10))
        

SyntaxError: unexpected EOF while parsing (<ipython-input-44-922528043d20>, line 16)